<a href="https://colab.research.google.com/github/TomashWue/ws22_dl_text_classification_hausarbeit/blob/convnet_hugging_face/bert_gnad10_ft_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importe

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import datasets
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import StratifiedKFold
from keras.optimizers import Adam

# 1. Lade den Datensatz

> Der Datensatz enthält ca. 10.000 Nachrichtenartikel zu 9 möglichen Themen. Das heißt, es gibt 9 mögliche Klassen/Label.

In [3]:
dataset = datasets.load_dataset('gnad10')

  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
train_texts = dataset['train']
test_texts = dataset['test']

# 2. Vorbereitungen

## 2.1 Initialisiere den Tokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

In [6]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"], padding=True, max_length=128, truncation=True)


train_dataset = train_texts.map(tokenize_dataset)
test_dataset = test_texts.map(tokenize_dataset)

## 2.2 Definiere die Parameter für die `Cross-Validation`

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 3. Führe das Training mit Cross-Validation aus

## 3.1 Lade das `pretrained model`

In [8]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-german-cased', num_labels=9)
model.compile(optimizer=Adam(learning_rate=1e-5), metrics='accuracy')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


## 3.2 `Cross-Validation` Schleife

In [9]:
for fold, (train_indices, val_indices) in enumerate(skf.split(train_dataset, train_dataset['label'])):

  # Bildschirmanzeige
  print(f'\nFold: {fold+1}'\n)
  
  # Trenne den Datensatz in trainings- und validation-Datensätze
  train_data = train_dataset.select(train_indices)
  val_data = train_dataset.select(val_indices)

  # Konvertiere die Datensätze in tensorflow-Datensätze
  train_tf_dataset = model.prepare_tf_dataset(train_data, batch_size=4, shuffle=True, tokenizer=tokenizer)
  val_tf_dataset = model.prepare_tf_dataset(val_data, batch_size=4, shuffle=False, tokenizer=tokenizer)
  
  # Train the model
  model.fit(train_tf_dataset, validation_data=val_tf_dataset, epochs=2, verbose=1)

  # Evaluate the model on the test data in each fold
  test_tf_dataset = model.prepare_tf_dataset(test_dataset, batch_size=4, shuffle=False, tokenizer=tokenizer)
  model.evaluate(test_tf_dataset)
  print(f'-----------------------')

Fold: 1
Epoch 1/2
1849/1849 [==============================] - 322s 158ms/step - loss: 0.6317 - accuracy: 0.8076 - val_loss: 0.3562 - val_accuracy: 0.8853
Epoch 2/2
257/257 [==============================] - 17s 54ms/step - loss: 0.3724 - accuracy: 0.8881
-----------------------
Fold: 2
Epoch 1/2
1849/1849 [==============================] - 311s 168ms/step - loss: 0.2014 - accuracy: 0.9381 - val_loss: 0.1118 - val_accuracy: 0.9708
Epoch 2/2
257/257 [==============================] - 14s 54ms/step - loss: 0.4480 - accuracy: 0.8862
-----------------------
Fold: 3
Epoch 1/2
1849/1849 [==============================] - 289s 156ms/step - loss: 0.0882 - accuracy: 0.9740 - val_loss: 0.0393 - val_accuracy: 0.9881
Epoch 2/2
257/257 [==============================] - 14s 53ms/step - loss: 0.5158 - accuracy: 0.8891
-----------------------
Fold: 4
Epoch 1/2
1849/1849 [==============================] - 306s 165ms/step - loss: 0.0389 - accuracy: 0.9888 - val_loss: 0.0322 - val_accuracy: 0.9886
Epoch